# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.

In [3]:
DATA_FOLDER = '../../ADA2017-Tutorials/02 - Intro to Pandas/Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [78]:
import os
import re
reg_exp = re.compile('([a-z]+)_data')
pd.set_option('display.max_columns', None)

directories = [x[1] for x in os.walk(DATA_FOLDER + '/ebola')][0]
country_names = [reg_exp.match(d).group(1) for d in directories]

countries = []

for c in country_names:
    country_frames = []
    all_files = glob.glob(DATA_FOLDER + '/ebola/' + c + '_data/' + r'*.csv')
    for f in all_files:
        new_frame = pd.read_csv(f)
        new_frame.columns = [x.lower() for x in new_frame.columns]
        country_frames.append(new_frame) 
        
    country_frame = pd.concat(country_frames)
    country_frame['country'] = c
    countries.append(country_frame)
    
result = pd.concat(countries, ignore_index=True)
result['date'] = pd.to_datetime(result.date)
result['national'] = pd.concat([result['national'].dropna(), result['totals'].dropna()])
result = result.drop('totals', axis=1)
result['variable'] = pd.concat([result['variable'].dropna(), result['description'].dropna()])
result = result.drop('description', axis=1)
result[['variable', 'date', 'country', 'national']]
all_other_columns = result.columns.difference(['variable', 'date', 'country', 'national'])
result[all_other_columns].sum(axis=1)

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      0.0
11      0.0
12      0.0
13      0.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
       ... 
7098    0.0
7099    0.0
7100    0.0
7101    0.0
7102    0.0
7103    0.0
7104    0.0
7105    0.0
7106    0.0
7107    0.0
7108    0.0
7109    0.0
7110    0.0
7111    0.0
7112    0.0
7113    0.0
7114    0.0
7115    0.0
7116    0.0
7117    0.0
7118    0.0
7119    0.0
7120    0.0
7121    0.0
7122    0.0
7123    0.0
7124    0.0
7125    0.0
7126    0.0
7127    0.0
Length: 7128, dtype: float64

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here